In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

In [ ]:
def replace_zeros(df):
    """Заменяет нули на моду, исключая 0 как моду."""
    for col in df.select_dtypes(include=np.number):
        counts = df[col].value_counts()
        mode = counts.drop(0, errors='ignore').nlargest(1).index[0] if len(counts)>1 else df[col].mean()
        df[col] = df[col].replace(0, mode)
    return df

(1) Выполните масштабирование данных.

In [ ]:
data = pd.read_csv('diabetes.csv')

X = data.drop('Outcome', axis=1)
y = data['Outcome']

X = replace_zeros(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Балансировка данных
ros = RandomOverSampler(random_state=294)
X_resampled, y_resampled = ros.fit_resample(X_scaled, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=294)

(2) Для построения ансамбля в качестве базового метода используйте поочередно изученные ранее методы с наилучшими гиперпараметрами.

In [ ]:
from sklearn.ensemble import BaggingClassifier

best_svm = SVC(C=10)
best_knn = KNeighborsClassifier(n_neighbors=11)
best_tree = DecisionTreeClassifier(max_depth=3)

# Создание бэггинговых ансамблей для каждой базовой модели

bagging_svm = BaggingClassifier(estimator=best_svm, n_estimators=10, random_state=294)
bagging_knn = BaggingClassifier(estimator=best_knn, n_estimators=10, random_state=294)
bagging_tree = BaggingClassifier(estimator=best_tree, n_estimators=10, random_state=294)


#Обучение ансамблей
bagging_svm.fit(X_train, y_train)
bagging_knn.fit(X_train, y_train)
bagging_tree.fit(X_train, y_train)


#Оценка моделей
print("\nBagging SVM:")
print(classification_report(y_test, bagging_svm.predict(X_test)))

print("\nBagging KNN:")
print(classification_report(y_test, bagging_knn.predict(X_test)))

print("\nBagging Decision Tree:")
print(classification_report(y_test, bagging_tree.predict(X_test)))


Bagging SVM:
              precision    recall  f1-score   support

           0       0.78      0.79      0.78       104
           1       0.77      0.76      0.76        96

    accuracy                           0.78       200
   macro avg       0.77      0.77      0.77       200
weighted avg       0.77      0.78      0.77       200


Bagging KNN:
              precision    recall  f1-score   support

           0       0.77      0.76      0.77       104
           1       0.74      0.76      0.75        96

    accuracy                           0.76       200
   macro avg       0.76      0.76      0.76       200
weighted avg       0.76      0.76      0.76       200


Bagging Decision Tree:
              precision    recall  f1-score   support

           0       0.84      0.67      0.75       104
           1       0.71      0.86      0.78        96

    accuracy                           0.77       200
   macro avg       0.78      0.77      0.76       200
weighted avg       0.7

(3) Получите структуру построенной модели.

In [ ]:
print(bagging_svm.estimator)
print(bagging_knn.estimator)
print(bagging_tree.estimator)

SVC(C=10)
KNeighborsClassifier(n_neighbors=11)
DecisionTreeClassifier(max_depth=3)


(4) Возьмите за основу один из базовых алгоритмов и определите оптимальное количество базовых алгоритмов в ансамбле (GridSearchCV)

In [ ]:
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

#Диапазон значений n_estimators для поиска
param_grid_bagging = {'n_estimators': range(1, 30, 1)} # от 1 до 30 с шагом 1


#Создание BaggingClassifier
bagging_tree = BaggingClassifier(estimator=best_tree, random_state=42)


#Использование GridSearchCV для подбора n_estimators
grid_bagging = GridSearchCV(bagging_tree, param_grid_bagging, cv=5, scoring='precision', n_jobs=-1) #n_jobs=-1 использует все ядра процессора
grid_bagging.fit(X_train, y_train)

#Лучшие параметры
print("Лучшее количество базовых алгоритмов:", grid_bagging.best_params_)
print("Лучшая точность:", grid_bagging.best_score_)


#Оценка лучшей модели на тестовом наборе
best_bagging_model = grid_bagging.best_estimator_
y_pred = best_bagging_model.predict(X_test)
print("\nОтчет о классификации на тестовом наборе:")
print(classification_report(y_test, y_pred))

Лучшее количество базовых алгоритмов: {'n_estimators': 12}
Лучшая точность: 0.7609870561345009

Отчет о классификации на тестовом наборе:
              precision    recall  f1-score   support

           0       0.76      0.75      0.76       104
           1       0.73      0.75      0.74        96

    accuracy                           0.75       200
   macro avg       0.75      0.75      0.75       200
weighted avg       0.75      0.75      0.75       200



Вывод: Дальнейшее увеличение числа алгоритмов не увеличивает качество